# Justin's Pyecca Point-Cloud Alignment (JPPCA)

Uses Casadi to optimize the transformation matrix T that minizies pointcloud measurement error. Uses pyecca to optimize the lie algebra se(3) elements.

# Problem Definition
Feature location definition:
$ \\ P_k = \begin{bmatrix}  x \\ y \\ z \\ 1 \end{bmatrix} \approx p_{k,0}$

Measurement of feature $k$ at pose $j$:
$ \\ p_{k,j} $

Transformation from pose $j$ to $j+1$:
$ \\ T_{j,j+1} $

Measurement error definition between poses $j$ and $j+1$ for feature $k$:
$ \\ e_{k,j+1} = p_{k,j+1} - T_{j,j+1}*p_{k,j} $

Cost function $J$:
$ \\ J(T_{j,j+1}) = \sum\limits_{k} e_{k,j+1}^T*Q*e_{k,j+1}$

So for pose $0$ to pose $1$:
$ \\ J(T_{0,1}) = \sum\limits_{k} e_{k,1}^T*Q*e_{k,1} = \sum\limits_{k} (p_{k,1} - T_{0,1}*p_{k,0})^T*Q*(p_{k,1} - T_{0,1}*p_{k,0})$

In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
%matplotlib qt
import numpy as np
import casadi as ca
import time
import BF_PCA

In [2]:
# Define symbolic variables used for optimization
trans_sym = ca.vertcat(ca.SX.sym('u'), ca.SX.sym('v'), ca.SX.sym('w'))
angles_sym = ca.vertcat(ca.SX.sym('omega1'), ca.SX.sym('omega2'), ca.SX.sym('omega3'))
all_sym = ca.vertcat(trans_sym, angles_sym)
all_sym

SX([u, v, w, omega1, omega2, omega3])

In [3]:
# Generate test data

# Points observed from pose 0:
# points_0 = np.array([
#     [1, 0, 0],
#     [0, 1, 0],
#     [0, 0, 1],
#     [1, 0, 1],
#     [0, 1, 1],
#     [1, 1, 0],
#     [-2, 1, 0],
# ])
points_0 = np.array([
    [1, 2, 3],
    [1, 2, -3],
    [1, -2, 3],
    [1, -2, -3],
    [-1, 2, 3],
    [-1, 2, -3],
    [-1, -2, 3],
    [-1, -2, -3],
])

R_true = BF_PCA.euler2rot(0.1, 0.2, 0.3).T
t_true = np.array([[1],
                   [2],
                   [3],
                  ])
T_01_true = np.vstack([np.hstack([R_true, t_true]), np.array([[0, 0, 0, 1]])])
T_01_true

points_1 = BF_PCA.applyT(points_0, T_01_true)

print('T_01_true:', T_01_true)
print('points_0:', points_0)
print('points_1:', points_1)

T_01_true: [[ 0.97517033  0.0978434  -0.19866933  1.        ]
 [-0.03695701  0.95642509  0.28962948  2.        ]
 [ 0.21835066 -0.27509585  0.93629336  3.        ]
 [ 0.          0.          0.          1.        ]]
points_0: [[ 1  2  3]
 [ 1  2 -3]
 [ 1 -2  3]
 [ 1 -2 -3]
 [-1  2  3]
 [-1  2 -3]
 [-1 -2  3]
 [-1 -2 -3]]
points_1: [[ 1.57484912  4.74478159  5.47703906]
 [ 2.76686511  3.00700473 -0.14072112]
 [ 1.18347554  0.91908125  6.57742245]
 [ 2.37549153 -0.81869562  0.95966227]
 [-0.37549153  4.81869562  5.04033773]
 [ 0.81652446  3.08091875 -0.57742245]
 [-0.76686511  0.99299527  6.14072112]
 [ 0.42515088 -0.74478159  0.52296094]]


In [4]:
import sys
sys.path.insert(0, '../../')
from pyecca.lie import se3, so3, matrix_lie_group
SE3 = se3._SE3()
T_sym = SE3.exp(SE3.wedge(all_sym))
T_sym

SX(@1=1, @2=((sq(omega1)+sq(omega2))+sq(omega3)), @3=sqrt(@2), @4=1e-07, @5=(fabs(@3)<@4), @6=0.5, @7=-0.0416667, @8=0.00138889, @9=((@5?((@6+(@7*@2))+(@8*sq(@2))):0)+((!@5)?((@1-cos(@3))/@2):0)), @10=(@9*omega3), @11=(@9*omega2), @12=(fabs(@3)<@4), @13=-0.166667, @14=0.00833333, @15=((@12?((@1+(@13*@2))+(@14*sq(@2))):0)+((!@12)?(sin(@3)/@3):0)), @16=(@9*omega1), @17=(@9*omega1), @18=0, @19=(@9*omega3), @20=(@9*omega2), @21=((sq(omega1)+sq(omega2))+sq(omega3)), @22=sqrt(@21), @23=(fabs(@22)<@4), @24=((@1-((@23?((@1+(@13*@21))+(@14*sq(@21))):0)+((!@23)?(sin(@22)/@22):0)))/@21), @25=(@24*omega3), @26=(@24*omega2), @27=(fabs(@22)<@4), @28=((@27?((@6+(@7*@21))+(@8*sq(@21))):0)+((!@27)?((@1-cos(@22))/@21):0)), @29=(@24*omega1), @30=(@24*omega3), @31=(@24*omega1), @32=(@24*omega2), 
[[(@1-((@10*omega3)+(@11*omega2))), ((@11*omega1)-(@15*omega3)), ((@15*omega2)+(@10*omega1)), ((((@1-((@25*omega3)+(@26*omega2)))*u)+(((@26*omega1)-(@28*omega3))*v))+(((@28*omega2)+(@25*omega1))*w))], 
 [((@15*om

In [5]:
# (8.95) Solution
# Build casadi cost function
J = BF_PCA.build_cost_pyecca(
            points_0=points_0,
            points_1=points_1,
            all_sym=all_sym)

# Setup and run optimizer
# Symbols/expressions
nlp = {}                 # NLP declaration
nlp['x'] = all_sym       # decision vars
nlp['f'] = J      # objective
nlp['g'] = 0             # constraints

# Create solver instance
# opts = {'ipopt.print_level':0, 'print_time':0, 'ipopt.sb': 'yes'}
# F = ca.nlpsol('F','ipopt',nlp,opts);
F = ca.nlpsol('F','ipopt',nlp);

# Solve the problem using a guess
# This uses original landmark/measure association (associates which landmark we think the measurement is measuring)
epsilon = 1e-20
x_input = np.array([epsilon,epsilon,epsilon,epsilon,epsilon,epsilon])
optim = F(x0=x_input)


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       21

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equa

In [6]:
SE3 = se3._SE3()
T_01_hat = SE3.exp(SE3.wedge(np.array(optim['x'])))

# points_1 = BF_PCA.applyT(points_0, T_01_hat)
# points_1

In [7]:
T_01_true

array([[ 0.97517033,  0.0978434 , -0.19866933,  1.        ],
       [-0.03695701,  0.95642509,  0.28962948,  2.        ],
       [ 0.21835066, -0.27509585,  0.93629336,  3.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [8]:
T_01_hat

SX(@1=0, 
[[0.97517, 0.0978434, -0.198669, 1], 
 [-0.036957, 0.956425, 0.289629, 2], 
 [0.218351, -0.275096, 0.936293, 3], 
 [@1, @1, @1, 1]])

In [9]:
float(optim['f'])

3.777900476194128e-23

In [22]:
# Barfoot method

# Initialize Top
Top = SE3.exp(SE3.wedge(np.array([0,0,0,0,0,0])))

for lcv in range(20):

    # (8.95) Solution
    # Build casadi cost function
    J = BF_PCA.build_cost_pyecca_barfoot(
                points_0=points_0,
                points_1=points_1,
                Top=Top,
                all_sym=all_sym)

    # Setup and run optimizer
    # Symbols/expressions
    nlp = {}                 # NLP declaration
    nlp['x'] = all_sym       # decision vars
    nlp['f'] = J      # objective
    nlp['g'] = 0             # constraints

    # Create solver instance
    # opts = {'ipopt.print_level':0, 'print_time':0, 'ipopt.sb': 'yes'}
    # F = ca.nlpsol('F','ipopt',nlp,opts);
    F = ca.nlpsol('F','ipopt',nlp);

    # Solve the problem using a guess
    # This uses original landmark/measure association (associates which landmark we think the measurement is measuring)
    epsilon = 1e-20
    x_input = np.array([epsilon,epsilon,epsilon,epsilon,epsilon,epsilon])
    optim = F(x0=x_input)

    That = SE3.exp(SE3.wedge(optim['x']))@Top

    Top = That

This is Ipopt version 3.12.3, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       15

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        1
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  

In [23]:
That = SE3.exp(SE3.wedge(optim['x']))@Top
That

SX(@1=0, @2=1, 
[[0.97517, 0.0978434, -0.198669, @2], 
 [-0.036957, 0.956425, 0.289629, 2], 
 [0.218351, -0.275096, 0.936293, 3], 
 [@1, @1, @1, @2]])

In [24]:
T_01_true

array([[ 0.97517033,  0.0978434 , -0.19866933,  1.        ],
       [-0.03695701,  0.95642509,  0.28962948,  2.        ],
       [ 0.21835066, -0.27509585,  0.93629336,  3.        ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])